In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark



In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("Colab").config('spark.ui.port', '4050')\
        .getOrCreate()

In [5]:
spark

In [6]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-11-16 04:50:30--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  7.84MB/s    in 1.7s    

2021-11-16 04:50:31 (7.84 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[],"uri":"/api/tunnels"}


In [7]:
ls drive/MyDrive/spark/Telco-Customer-Churn.csv

drive/MyDrive/spark/Telco-Customer-Churn.csv


In [8]:
df = spark.read.csv("drive/MyDrive/spark/Telco-Customer-Churn.csv", header=True, inferSchema=True)

In [9]:
df.show(5)
df.count()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

7043

In [10]:
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



In [11]:
df.describe().show()

+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|summary|customerID|gender|     SeniorCitizen|Partner|Dependents|            tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|    MonthlyCharges|      TotalCharges|Churn|
+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|  count|      7043|  7043|              7043|   7043|      7043|     

In [12]:
df1 = df.drop("customerID") 
df1.printSchema()

root
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



In [13]:
from pyspark.sql import functions as F
df1.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df1.columns]).show()

+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0|            0|             0|          11|   

In [14]:
 df1.show(5)

+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|         No|             No|Month-to-month|     

In [15]:
from pyspark.sql.functions import when, lit

In [16]:
df1.select("PaperlessBilling").distinct().show()

+----------------+
|PaperlessBilling|
+----------------+
|              No|
|             Yes|
+----------------+



In [17]:
df1.printSchema()

root
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



In [18]:
df1 = df1.withColumn('Partner', when(df1.Partner=='Yes', 
lit(1)).otherwise(0))
df1 = df1.withColumn('DeviceProtection', when(df1.DeviceProtection=='Yes', 
lit(1)).when(df1.DeviceProtection=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('gender', when(df1.gender=='Female', 
lit(1)).otherwise(0))
df1 = df1.withColumn('Dependents', when(df1.Dependents=='Yes', 
lit(1)).otherwise(0))
df1 = df1.withColumn('PhoneService', when(df1.PhoneService=='Yes', 
lit(1)).otherwise(0))
df1 = df1.withColumn('MultipleLines', when(df1.MultipleLines=='Yes', 
lit(1)).when(df1.DeviceProtection=='No phone service',lit(-1)).otherwise(0))
df1 = df1.withColumn('InternetService', when(df1.InternetService=='Fiber optic', 
lit(1)).when(df1.InternetService=='DSL',lit(2)).otherwise(0))
df1 = df1.withColumn('OnlineSecurity', when(df1.OnlineSecurity=='Yes', 
lit(1)).when(df1.OnlineSecurity=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('OnlineBackup', when(df1.OnlineBackup=='Yes', 
lit(1)).when(df1.OnlineBackup=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('TechSupport', when(df1.TechSupport=='Yes', 
lit(1)).when(df1.TechSupport=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('Churn', when(df1.Churn=='Yes', 
lit(1)).otherwise(0))
df1 = df1.withColumn('StreamingTV', when(df1.StreamingTV=='Yes', 
lit(1)).when(df1.StreamingTV=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('StreamingMovies', when(df1.StreamingMovies=='Yes', 
lit(1)).when(df1.StreamingMovies=='No internet Service',lit(-1)).otherwise(0))
df1 = df1.withColumn('PaperlessBilling', when(df1.PaperlessBilling=='Yes', 
lit(1)).otherwise(0))
df1 = df1.withColumn('Contract', when(df1.Contract=='Month-to-month', 
lit(0)).when(df1.Contract=='One year',lit(1)).otherwise(2))


In [19]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

df_stats = df.select(
    _mean(col('TotalCharges')).alias('mean')
).collect()

mean = df_stats[0]['mean']
df1 = df1.na.fill(value=mean)

In [20]:
df1.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df1.columns]).show()

+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0|            0|             0|           0|   

In [21]:
df1.show(5)

+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+--------------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+--------------------+--------------+------------+-----+
|     1|            0|      1|         0|     1|           0|            0|              2|             0|           1|               0|          0|          0|              0|       0|               1|    Electronic check|   

In [22]:
df1.describe().show()

+-------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+
|summary|            gender|     SeniorCitizen|            Partner|        Dependents|            tenure|       PhoneService|      MultipleLines|   InternetService|    OnlineSecurity|      OnlineBackup|   DeviceProtection|       TechSupport|        StreamingTV|   StreamingMovies|          Contract|  PaperlessBilling|       PaymentMethod|    MonthlyCharges|      TotalCharges|              Churn|
+-------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+--

In [60]:
x = df1.drop('customerID','PhoneService','OnlineBackup','TechSupport','DeviceProtection','StreamingMovies','PaymentMethod','Dependents')
#y = df1.select('Churn')

In [42]:
x.columns

['gender',
 'SeniorCitizen',
 'Partner',
 'tenure',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'StreamingTV',
 'Contract',
 'PaperlessBilling',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [61]:
from pyspark.sql.types import IntegerType
df1 = df1.withColumn("TotalCharges", df1["TotalCharges"].cast(IntegerType()))

In [44]:
x.printSchema()

root
 |-- gender: integer (nullable = false)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: integer (nullable = false)
 |-- tenure: integer (nullable = true)
 |-- MultipleLines: integer (nullable = false)
 |-- InternetService: integer (nullable = false)
 |-- OnlineSecurity: integer (nullable = false)
 |-- StreamingTV: integer (nullable = false)
 |-- Contract: integer (nullable = false)
 |-- PaperlessBilling: integer (nullable = false)
 |-- MonthlyCharges: double (nullable = false)
 |-- TotalCharges: integer (nullable = true)
 |-- Churn: integer (nullable = false)



In [45]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import MinMaxScaler


In [69]:
feature_names = ['gender',
 'SeniorCitizen',
 'Partner',
 'tenure',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'StreamingTV',
 'Contract',
 'PaperlessBilling',
 'MonthlyCharges',
 'TotalCharges']
assembler = VectorAssembler(inputCols=feature_names, outputCol="features")
scalar = MinMaxScaler(inputCol="features ", outputCol="Churn")
x = assembler.transform(x)

In [47]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator


In [48]:
from pyspark.ml.classification import LogisticRegression


In [70]:
train, test = x.randomSplit([0.8, 0.2], seed = 2018)
#train_y, test_y = y.randomSplit([0.8, 0.2], seed = 2018)

In [71]:
lr = LogisticRegression(maxIter=1000, labelCol='Churn')

In [72]:
model = lr.fit(train)

In [73]:
from pyspark.ml import Pipeline 
pipeline = Pipeline(stages = [lr])

In [74]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [.1, .01]).build()

In [75]:
val = CrossValidator(estimator=pipeline, estimatorParamMaps=BinaryClassificationEvaluator(), numFolds = 2)

In [76]:
prediction = model.transform(test)

In [77]:
evaluator = BinaryClassificationEvaluator(labelCol='Churn')


In [78]:
print(evaluator.evaluate(prediction))

0.8392776149123222


In [79]:
prediction.select('Churn','prediction', 'probability').show(20)

+-----+----------+--------------------+
|Churn|prediction|         probability|
+-----+----------+--------------------+
|    0|       0.0|[0.91150632199829...|
|    0|       0.0|[0.75844312893352...|
|    0|       0.0|[0.75795001100921...|
|    0|       0.0|[0.75762088529434...|
|    0|       0.0|[0.75729145535872...|
|    0|       0.0|[0.75696172131074...|
|    1|       0.0|[0.75679674027879...|
|    0|       0.0|[0.75591958389235...|
|    0|       0.0|[0.75591958389235...|
|    1|       0.0|[0.75591958389235...|
|    1|       0.0|[0.75558858759577...|
|    0|       0.0|[0.75542297561462...|
|    1|       0.0|[0.75525728776409...|
|    1|       0.0|[0.67905411502129...|
|    1|       0.0|[0.67723243731212...|
|    0|       0.0|[0.67585905020409...|
|    0|       0.0|[0.67546609570533...|
|    1|       0.0|[0.67402794295140...|
|    1|       0.0|[0.67402794295140...|
|    1|       0.0|[0.67363384055183...|
+-----+----------+--------------------+
only showing top 20 rows

